In [0]:
import os
import pdb

proj_folder = '/content/gpt-2'
git_src = 'https://github.com/openai/gpt-2' 
if not os.path.exists(proj_folder):
  !git clone $git_src
else:
  print('existed: %s' % proj_folder)
  os.chdir(proj_folder)  
  !git pull origin master

os.chdir(proj_folder)
!pip3 install -r requirements.txt

In [0]:
import sys
import tensorflow as tf

print('tf version: %s' % tf.__version__)

device_name = tf.test.gpu_device_name()
if 'GPU' in device_name:
  print('GPU ready: %s' % device_name) 
  GPU_FLAG = True
else:
  print('CPU only.....')    

src_path = '/content/gpt-2/src'
if src_path not in sys.path:
  sys.path += [src_path]

os.chdir(proj_folder)
model_name= '345M'  
if os.path.exists(os.path.join('models', model_name)) == False:
  print('download model %s....' % model_name)
  !PYTHONPATH=src; python ./download_model.py $model_name
else:
  print('existed: model %s' % model_name)    

In [0]:
# the following code is copied from: 
# https://stackoverflow.com/questions/25010369/wget-curl-large-file-from-google-drive/39225039#39225039

import requests

def download_file_from_google_drive(id, destination):
    def get_confirm_token(response):
        for key, value in response.cookies.items():
            if key.startswith('download_warning'):
                return value

        return None

    def save_response_content(response, destination):
        CHUNK_SIZE = 32768

        with open(destination, "wb") as f:
            for chunk in response.iter_content(CHUNK_SIZE):
                if chunk: # filter out keep-alive new chunks
                    f.write(chunk)

    URL = "https://docs.google.com/uc?export=download"
    session = requests.Session()
    response = session.get(URL, params = { 'id' : id }, stream = True)
    token = get_confirm_token(response)
    if token:
        params = { 'id' : id, 'confirm' : token }
        response = session.get(URL, params = params, stream = True)
    save_response_content(response, destination)    

In [0]:
# donwload fine-tuned model for patents

ckpt_path = 'saved_checkpoint'
if os.path.exists(ckpt_path):
  print('Existed: %s' % ckpt_path)
  !ls $ckpt_path
else:
  os.mkdir(ckpt_path)
  os.chdir(ckpt_path)
  print('Downloading files to %s....' % ckpt_path)
  download_file_from_google_drive('10Qc8SIuwq6w6guwDnpdNzKH24NxZ3oG-', 'checkpoint')
  download_file_from_google_drive('1KDO8ikS5IJqo1S6zfbOnFae3-zS6-8f0', 'model-945000.meta')
  download_file_from_google_drive('1Z0PnW0BHFApZBBWL3SGDzwgFme1mwFwi', 'model-945000.data-00000-of-00001')
  download_file_from_google_drive('1ipOdKVOUM8h45njLLtbhJFbfqTNFLNq_', 'model-945000.index')
  !ls -al 
  print('Download: ok')
os.chdir(proj_folder)

In [0]:
# the following is my enchancement based on: 
# https://github.com/openai/gpt-2/blob/master/src/sample.py
# https://github.com/openai/gpt-2/blob/master/src/interactive_conditional_samples.py

import json
import os
import numpy as np

import model, sample, encoder

def generate_text(seed_text, sess, output, context, nsamples, batch_size):
    generated = 0
    for _ in range(nsamples // batch_size):
      out = sess.run(output, feed_dict={
          context: [context_tokens for _ in range(batch_size)]
      })[:, len(context_tokens):]
      for i in range(batch_size):
        generated += 1
        text = enc.decode(out[i])
        text = till_end_of_text(text)
        text = seed_text + ' ' + text
        print("=" * 40 + " SAMPLE " + str(generated) + " " + "=" * 40)
        text = text.replace(tag_start, '').strip()
        span_text = text.replace(' @@@ ','\n    ')
        print(span_text)
    print("=" * 80)
    print('\n')

def dynamic_kp_logits(logits, top_kp):
    k = 1000 # observe probability of the top n 
    probs_logits = tf.nn.softmax(logits)
    k_probs, _ = tf.nn.top_k(probs_logits, k=k)
    k_probs = tf.squeeze(k_probs)
    probs_max = tf.reduce_max(k_probs)
    k_threshold = tf.multiply(probs_max, top_kp)
    probs_mask = tf.to_int32(k_probs >= k_threshold)
    num_of_k = tf.count_nonzero(probs_mask, dtype=tf.float32) 

    values, _ = tf.nn.top_k(logits, k=tf.cast(num_of_k, dtype=tf.int32))
    min_values = values[:, -1, tf.newaxis]
    result_logits = tf.where(
        logits < min_values,
        tf.ones_like(logits, dtype=logits.dtype) * -1e10,
        logits,)

    return result_logits

def sample_sequence_kp(*, hparams, length, start_token=None, batch_size=None, 
                       context=None, temperature=1, top_kp=0.1):
    if start_token is None:
        assert context is not None, 'Specify exactly one of start_token and context!'
    else:
        assert context is None, 'Specify exactly one of start_token and context!'
        context = tf.fill([batch_size, 1], start_token)

    def step(hparams, tokens, past=None):
        lm_output = model.model(hparams=hparams, X=tokens, past=past, reuse=tf.AUTO_REUSE)

        logits = lm_output['logits'][:, :, :hparams.n_vocab]
        presents = lm_output['present']
        presents.set_shape(model.past_shape(hparams=hparams, batch_size=batch_size))
        return {
            'logits': logits,
            'presents': presents,
        }

    with tf.name_scope('sample_sequence'):
        def body(past, prev, output):
            next_outputs = step(hparams, prev, past=past)
            logits = next_outputs['logits'][:, -1, :]  / tf.to_float(temperature)
            logits = dynamic_kp_logits(logits, top_kp)
            samples = tf.multinomial(logits, num_samples=1, output_dtype=tf.int32)
            return [
                next_outputs['presents'] if past is None else tf.concat([past, next_outputs['presents']], axis=-2),
                samples,
                tf.concat([output, samples], axis=1)
            ]

        past, prev, output = body(None, context, context)

        def cond(*args):
            return True

        _, _, tokens = tf.while_loop(
            cond=cond, body=body,
            maximum_iterations=length - 1,
            loop_vars=[
                past,
                prev,
                output
            ],
            shape_invariants=[
                tf.TensorShape(model.past_shape(hparams=hparams, batch_size=batch_size)),
                tf.TensorShape([batch_size, None]),
                tf.TensorShape([batch_size, None]),
            ],
            back_prop=False,
        )

        return tokens

def till_end_of_text(text):
  tag_end = '<|endoftext|>'
  result = ''
  pos2 = text.find(tag_end)
  if pos2 == -1:
    result = '(unable to generate....probably not patent text)'
  elif pos2 == 0:
    result = '(end of patent claim)'
  else:
    result = text[:pos2].strip()

  return result

# main program
model_name='345M'
seed=None
nsamples=1
batch_size=1
length=None
temperature=1
top_k=40
top_p=0.9
top_kp=0.1

tag_start = '<|startoftext|>'

models_dir = 'models'
models_dir = os.path.expanduser(os.path.expandvars(models_dir))
if batch_size is None:
  batch_size = 1
assert nsamples % batch_size == 0

enc = encoder.get_encoder(model_name, models_dir)
hparams = model.default_hparams()
with open(os.path.join(models_dir, model_name, 'hparams.json')) as f:
  hparams.override_from_dict(json.load(f))

if length is None:
  length = hparams.n_ctx // 2
elif length > hparams.n_ctx:
  raise ValueError("Can't get samples longer than window size: %s" % hparams.n_ctx)

with tf.Session(graph=tf.Graph()) as sess:
  context = tf.placeholder(tf.int32, [batch_size, None])
  np.random.seed(seed)
  tf.set_random_seed(seed)

  # original sampling in GPT-2
  output_top_k = sample.sample_sequence(
    hparams=hparams, length=length,
    context=context,
    batch_size=batch_size,
    temperature=temperature, top_k=top_k, top_p=1)
  output_top_p = sample.sample_sequence(
    hparams=hparams, length=length,
    context=context,
    batch_size=batch_size,
    temperature=temperature, top_k=0, top_p=top_p)
  
  # a different sampling in our research
  output_top_kp = sample_sequence_kp(
    hparams=hparams, length=length,
    context=context,
    batch_size=batch_size,
    temperature=temperature, top_kp=top_kp)

  saver = tf.train.Saver()

  # use the fine-tuned model for patents
  ckpt = tf.train.latest_checkpoint(ckpt_path)

  # original model released by OpenAI
  # ckpt = tf.train.latest_checkpoint(os.path.join(models_dir, model_name))

  saver.restore(sess, ckpt)
  while True:
    print('Input text or "exit" or "Enter" key for unconditional sampling.....')
    seed_text = input(">>> ")
    if seed_text == 'exit':
      break
    if seed_text == '':
      seed_text = tag_start 
    context_tokens = enc.encode(seed_text)
    print('top_k = %s' % top_k)
    generate_text(seed_text, sess, output_top_k, context, nsamples, batch_size)
    print('top_p = %s' % top_p)
    generate_text(seed_text, sess, output_top_p, context, nsamples, batch_size)
    print('top_kp = %s' % top_kp)
    generate_text(seed_text, sess, output_top_kp, context, nsamples, batch_size)

  print('Thank you.')